In [74]:
from minio import Minio

client = Minio(endpoint='localhost:9000',
               access_key="u4rWRgp0IIZ0dGQoMWRZ",
               secret_key="GYgeifrnYK9q9o2BGED2k3NroASmCvou59PxYLjm",
               secure=False)

print("Total buckets: ", len(client.list_buckets()))

Total buckets:  2


In [75]:
import boto3
# Set up MinIO client
s3 = boto3.client('s3',
                  endpoint_url='http://localhost:9000',
                  aws_access_key_id="u4rWRgp0IIZ0dGQoMWRZ",
                  aws_secret_access_key="GYgeifrnYK9q9o2BGED2k3NroASmCvou59PxYLjm")
# Create a bucket
# s3.create_bucket(Bucket='mybucket')
# # Upload a file to the bucket
# s3.upload_file('myfile.txt', 'mybucket', 'myfile.txt')
# List objects in the bucket
response = s3.list_objects(Bucket='datalake')
for obj in response.get('Contents', []):
    print(obj['Key'])

cleaned/sales/sales_data.parquet
raw/sales/sales_data.csv


In [81]:
import pandas as pd
from minio import Minio
from io import StringIO, BytesIO

# MinIO configurations
minio_client = Minio(endpoint='localhost:9000',
               access_key="u4rWRgp0IIZ0dGQoMWRZ",
               secret_key="GYgeifrnYK9q9o2BGED2k3NroASmCvou59PxYLjm",
               secure=False)

print("Total buckets: ", len(client.list_buckets()))

#-------------------------------------- E T L -------------------------------------------------------#

def extract_data_from_minio(bucket_name, object_name):
    # Function to read data from MinIO
    obj = minio_client.get_object(bucket_name, object_name)
    data = obj.data.decode('utf-8')
    return data

def transform_data(data):
    def rename_columns(df):
        """
        Rename columns of a DataFrame to desired format.
        
        Args:
        - df: pandas DataFrame containing the data
        
        Returns:
        - df: pandas DataFrame with renamed columns
        """
        column_mapping = {
            'Unnamed: 0': 'id', 
            'ProductCategoryName': 'product_category_name', 
            'ProductSubcategoryName': 'product_sub_category_name',
            'ProductName': 'product_name', 
            'SalesTerritoryCountry': 'sales_territory_country', 
            'SalesAmount': 'sale_amount', 
            'OrderDate': 'orderdate'
        }
        
        return df.rename(columns=column_mapping)   
    def convert_data_types(df):
        """
        Convert columns of a DataFrame to desired data types.
        
        Args:
        - df: pandas DataFrame containing the data
        
        Returns:
        - df: pandas DataFrame with columns converted to desired data types
        """
        df['product_category_name'] = df['product_category_name'].astype(str)
        df['product_name'] = df['product_name'].astype(str)
        df['product_sub_category_name'] = df['product_sub_category_name'].astype(str)
        df['sales_territory_country'] = df['sales_territory_country'].astype(str)
        df['sale_amount'] = df['sale_amount'].astype(float)
        
        return df

    # Function to transform data using pandas
    # Example transformation, replace 'your_transformation_logic' with your actual logic
    df = pd.read_csv(StringIO(data), header=0)  # Assuming data is CSV, change as needed
    
    renamed_df = rename_columns(df)
    converted_df = convert_data_types(renamed_df)

    cleaned_df = converted_df
    
    return cleaned_df

def load_data_to_minio(cleaned_data, bucket_name, object_name):
    # Function to save cleaned data to MinIO in Parquet format
    with BytesIO() as f:
        cleaned_data.to_parquet(f,index=False, engine='pyarrow')
        f.seek(0)
        minio_client.put_object(bucket_name, object_name, f, f.getbuffer().nbytes)

#-----------------------------------------------------------------------------------------------#

if __name__ == "__main__":
    # Example usage
    bucket_name = 'datalake'
    raw_data_object_name = 'raw/sales/sales_data.csv'
    cleaned_data_object_name = 'cleaned/sales/sales_data.parquet'

    # Read raw data from MinIO
    raw_data = extract_data_from_minio(bucket_name, raw_data_object_name)

    # Transform data
    cleaned_data = transform_data(raw_data)

    # Save cleaned data to MinIO in Parquet format
    load_data_to_minio(cleaned_data, bucket_name, cleaned_data_object_name)


Total buckets:  2
0               Canada
1               France
2        United States
3        United States
4            Australia
             ...      
60393           Canada
60394           Canada
60395           Canada
60396           Canada
60397           Canada
Name: sales_territory_country, Length: 60398, dtype: object


In [77]:
def read_parquet_from_minio(bucket_name, object_name):
    # Function to read Parquet data from MinIO
    obj = minio_client.get_object(bucket_name, object_name)
    with BytesIO() as f:
        f.write(obj.data)
        f.seek(0)
        df = pd.read_parquet(f)
    return df
# Read cleaned Parquet data from MinIO and print
cleaned_df = read_parquet_from_minio(bucket_name, cleaned_data_object_name)
print(cleaned_df)

          id product_category_name product_sub_category_name  \
0          0                 Bikes                Road Bikes   
1          1                 Bikes            Mountain Bikes   
2          2                 Bikes            Mountain Bikes   
3          3                 Bikes                Road Bikes   
4          4                 Bikes            Mountain Bikes   
...      ...                   ...                       ...   
60393  60393           Accessories                   Fenders   
60394  60394              Clothing                      Caps   
60395  60395           Accessories                   Fenders   
60396  60396           Accessories               Bike Stands   
60397  60397              Clothing                      Caps   

                  product_name sales_territory_country  sale_amount  \
0             Road-150 Red, 62                  Canada    3578.2700   
1      Mountain-100 Silver, 44                  France    3399.9900   
2      Mountain-10